In [51]:
import sys
sys.path.append('../')
from pdf2df import convert_to_df
# df = convert_to_df('../../error_pdf/BPX -  I0759074.pdf')
df = convert_to_df('../../pdf_issue/bpx_1.pdf')
df

{'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Apache FOP Version 0.94', 'creationDate': "D:20220802150227+01'00'", 'modDate': "D:20220805204131+01'00'", 'trapped': '', 'encryption': None}


Deprecation: 'getText' removed from class 'Page' after v1.19 - use 'get_text'.


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,BPX,149,10,163,20,0,0,1
1,Electromechanical,165,10,222,20,0,0,1
2,Ltd.,224,10,236,20,0,0,1
3,t/a,238,10,245,20,0,0,1
4,BPX,247,10,262,20,0,0,1
...,...,...,...,...,...,...,...,...
4257,"4,995.22",746,478,777,489,57,0,11
4258,VAT,783,478,799,489,57,0,11
4259,"26,713.57",741,491,777,502,58,0,11
4260,Total,783,491,802,502,58,0,11


In [52]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [53]:


pos = find_text(df,'Quotation')
quotation_no = df[pos[0]+1:pos[0]+2]['TEXT'].values[0]
quotation_no

'QE009837'

In [54]:
pos_date =find_text(df,'date')[0]
date = df[pos_date+1:pos_date+2]['TEXT'].values[0]
date

'29/07/22'

In [55]:
def get_table_data(df):
    result = list()
    page_no = df['page_no'].unique()
    try:
        for page in page_no:
            mydf = df[df['page_no'] == page].reset_index(drop=True)
            get_result(mydf, result)
    except Exception as e:
        traceback.print_exc()

    return result

In [59]:
def get_result(df, result):
    table_start = find_next_word(df, 'Description','Price')
    table_data = df[table_start[0]:]
    msg = dict()
    for index, row in table_data.iterrows():
#     print(row['TEXT'])
        if row['X1'] < 40: 
            if len(msg.keys()) > 4:
                result.append(msg)
                msg = dict()
            msg['qty'] = row['TEXT']
            msg['part'] = ""
            
        if ('part' in msg) and (50 < row['X1'] < 80):
            msg['part'] = row['TEXT']
            msg['desc'] = ''

        if ('desc' in msg) and (250< row['X1'] < 450):
            if len(msg['desc'])<3:
                msg['desc'] = row['TEXT']
                msg['unit'] = 0
            else:
                msg['desc'] = msg['desc'] +" "+row['TEXT']


        if 'unit' in msg and (600 < row['X1'] < 650):
            msg['unit'] = row['TEXT']
            msg['disc'] = 0
            msg['total'] = 0

        if 'disc' in msg and 690 < row['X1'] < 700:
            msg['disc'] = row['TEXT']
    #         print(row['TEXT'])
            msg['total'] = 0

        if 'total' in msg and 730 < row['X1'] < 770:
            msg['total'] = row['TEXT']

    if len(msg.keys()) >4:
        result.append(msg)
        

In [60]:
result  = get_table_data(df)
result

[{'qty': '2',
  'part': '13975',
  'desc': 'SCHNEIDER KAEDRA 3 MODULE ENCLOSURE',
  'unit': '8.01',
  'disc': '30.00',
  'total': '11.21'},
 {'qty': '2',
  'part': '13441',
  'desc': 'SCHNEIDER - KAEDRA 4 MOD INS ENCLOSURE NO. SAREL - ENN82002)',
  'unit': '16.96',
  'disc': '30.00',
  'total': '23.74'},
 {'qty': '2',
  'part': '13442',
  'desc': 'SCHNEIDER KAEDRA 6 MOD INS ENCLOSURE SAREL ENN82003)',
  'unit': '25.04',
  'disc': '30.00',
  'total': '35.06'},
 {'qty': '2',
  'part': '13443',
  'desc': 'SCHNEIDER KAEDRA 8 MODULE ENCLOSURE',
  'unit': '32.17',
  'disc': '30.00',
  'total': '45.04'},
 {'qty': '2',
  'part': '13444',
  'desc': 'SCHNEIDER Kaedra 12 Mod Ins Enclosure IP65',
  'unit': '38.10',
  'disc': '30.00',
  'total': '53.34'},
 {'qty': '2',
  'part': '13432',
  'desc': 'Schneider - Kaedra 1 row 18 mod ins enclosure',
  'unit': '71.57',
  'disc': '30.00',
  'total': '100.20'},
 {'qty': '2',
  'part': '13433',
  'desc': 'SCHNEIDER Kaedra 2 row 24 mod ins enclosure',
  'un

In [61]:
import pandas as pd
new_df = pd.DataFrame(result)
new_df

,qty,part,desc,unit,disc,total
0,2,13975,SCHNEIDER KAEDRA 3 MODULE ENCLOSURE,8.01,30.00,11.21
1,2,13441,SCHNEIDER - KAEDRA 4 MOD INS ENCLOSURE NO. SAR...,16.96,30.00,23.74
2,2,13442,SCHNEIDER KAEDRA 6 MOD INS ENCLOSURE SAREL ENN...,25.04,30.00,35.06
3,2,13443,SCHNEIDER KAEDRA 8 MODULE ENCLOSURE,32.17,30.00,45.04
4,2,13444,SCHNEIDER Kaedra 12 Mod Ins Enclosure IP65,38.10,30.00,53.34
...,...,...,...,...,...,...
114,10,A9XPH324,SCHNEIDER ACTI 9 COMB BUSBAR 3P 100A 24MOD.,42.17,50.00,210.85
115,10,A9XPH424,SCHNEIDER Acti9 Comb Busbar 4P 10,44.27,50.00,221.35
116,50,LP1SK0600BD,SCHNEIDER CONTACTOR,31.82,40.00,954.60
117,40,LA1SK10,SCHNEIDER CONTACT BLOCK 1 PWR POLE,11.53,40.00,276.72


In [39]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
#     print(row['TEXT'])
    if row['X1'] < 40: 
        if len(msg.keys()) > 4:
            result.append(msg)
            msg = dict()
            
        msg['qty'] = row['TEXT']
        msg['part'] = ""
#         print(msg)
    if ('part' in msg) and (50 < row['X1'] < 80):
        msg['part'] = row['TEXT']
        msg['desc'] = ''

    if ('desc' in msg) and (250< row['X1'] < 450):
        if len(msg['desc'])<3:
            msg['desc'] = row['TEXT']
            msg['unit'] = 0
        else:
            msg['desc'] = msg['desc'] +" "+row['TEXT']

        
    if 'unit' in msg and (600 < row['X1'] < 650):
        msg['unit'] = row['TEXT']
        msg['disc'] = 0
        msg['total'] = 0
        
    if 'disc' in msg and 690 < row['X1'] < 700:
        msg['disc'] = row['TEXT']
#         print(row['TEXT'])
        msg['total'] = 0
        
    if 'total' in msg and 730 < row['X1'] < 770:
        msg['total'] = row['TEXT']
    
if len(msg.keys()) >4:
    result.append(msg)

,qty,part,desc,unit,disc,total
0,2,13975,SCHNEIDER KAEDRA 3 MODULE ENCLOSURE,8.01,30.00,11.21
1,2,13441,SCHNEIDER - KAEDRA 4 MOD INS ENCLOSURE NO. SAR...,16.96,30.00,23.74
2,2,13442,SCHNEIDER KAEDRA 6 MOD INS ENCLOSURE SAREL ENN...,25.04,30.00,35.06
3,2,13443,SCHNEIDER KAEDRA 8 MODULE ENCLOSURE,32.17,30.00,45.04
4,2,13444,SCHNEIDER Kaedra 12 Mod Ins Enclosure IP65,38.10,30.00,53.34
5,2,13432,Schneider - Kaedra 1 row 18 mod ins enclosure,71.57,30.00,100.20
6,2,13433,SCHNEIDER Kaedra 2 row 24 mod ins enclosure,99.28,30.00,138.99
7,2,13435,SCHNEIDER KAEDRA 3 ROW 36 MOD INS ENCLOSURE,116.59,30.00,163.23
8,2,13436,Schneider KAEDRA ENCLOSURE 3X18 MOD,178.93,30.00,250.50
9,4,VCCF0,SCHNEIDER DISCONNECT SWITCH 25A 3P KIT AND SHA...,34.27,40.00,82.25


In [38]:
new_df['desc'].to_list()

['SCHNEIDER KAEDRA 3 MODULE ENCLOSURE',
 'SCHNEIDER - KAEDRA 4 MOD INS ENCLOSURE NO. SAREL - ENN82002)',
 'SCHNEIDER KAEDRA 6 MOD INS ENCLOSURE SAREL ENN82003)',
 'SCHNEIDER KAEDRA 8 MODULE ENCLOSURE',
 'SCHNEIDER Kaedra 12 Mod Ins Enclosure IP65',
 'Schneider - Kaedra 1 row 18 mod ins enclosure',
 'SCHNEIDER Kaedra 2 row 24 mod ins enclosure',
 'SCHNEIDER KAEDRA 3 ROW 36 MOD INS ENCLOSURE',
 'Schneider KAEDRA ENCLOSURE 3X18 MOD',
 'SCHNEIDER DISCONNECT SWITCH 25A 3P KIT AND SHAFT DOOR INTERLOCK PLATE AN D INCOMING SHROUD']